In [56]:
from os import listdir, system, path

"""Python WPS execute"""
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput
#wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps_url = "http://localhost:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [57]:
print wps.identification.title

for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract)

Flyingpigeon
subset_continents : 	 Returns only the selected polygon for each input dataset
subset_countries : 	 Returns only the selected polygon for each input dataset
subset_regionseurope : 	 Returns only the selected polygon for each input dataset
subset_points : 	 Extract Timeseries for specified coordinates from gridded datasets
indices_single : 	 This process calculates climate indices based on one single input variable.
weatherregimes_reanalyse : 	 Weather Regimes based on pressure patterns, fetching selected Realayses Datasets
weatherregimes_model : 	 Weather Regimes based on pressure patterns, fetching selected Realayses Datasets
weatherregimes_projection : 	 Weather Regimes detection based on trained reference statistics
analogs : 	 Search for days with analog pressure pattern
analogs_viewer : 	 Visualisation of text output of analogue process
segetalflora : 	 Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44
sdm_gbifsearch : 	 Speci

In [58]:
p = wps.describeprocess(identifier='weatherregimes_reanalyse')

# to recieve some informations uncomment the follwing lines

#for input in p.dataInputs:
#  printInputOutput(input)
#  print '\n'

In [59]:
training = wps.execute(
    identifier="weatherregimes_reanalyse", 
    inputs=[ ('season', 'DJF')], #('BBox','-80,22.5,50,70'),
    async=True) 

In [60]:
monitorExecution(training, sleepSecs=5)
print training.getStatus()

ProcessSucceeded


In [61]:
for o in training.processOutputs:
    print o.reference 

http://localhost:8090/wpsoutputs/flyingpigeon/output_pca-0573438a-4203-11e6-9e3e-417988cdf122.dat
http://localhost:8090/wpsoutputs/flyingpigeon/Routput_graphic-0573438a-4203-11e6-9e3e-417988cdf122.pdf
http://localhost:8090/wpsoutputs/flyingpigeon/output_classification-0573438a-4203-11e6-9e3e-417988cdf122.Rdat
http://localhost:8090/wpsoutputs/flyingpigeon/output_netcdf-0573438a-4203-11e6-9e3e-417988cdf122.nc


In [62]:
# Next step is to perform a projection with the trained output 
p = wps.describeprocess(identifier='weatherregimes_projection')

In [63]:
for input in p.dataInputs:
    printInputOutput(input)

 identifier=resource, title=Resource, abstract=NetCDF File, data type=ComplexData
 Supported Value: mimeType=application/x-netcdf, encoding=None, schema=None
 Default Value: mimeType=application/x-netcdf, encoding=None, schema=None 
 minOccurs=1, maxOccurs=1000
 identifier=Rdat, title=R - workspace, abstract=R workspace as outputed from weatherregieme reference process, data type=//www.w3.org/TR/xmlschema-2/#string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1
 identifier=dat, title=R - datafile, abstract=R datafile as outputed from weatherregieme reference process, data type=//www.w3.org/TR/xmlschema-2/#string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1
 identifier=netCDF, title=netCDF reference, abstract=netCDF file as outputed from weatherregieme reference process, data type=//www.w3.org/TR/xmlschema-2/#string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1
 identifier=season, title=Time region, abstract=Select the month

In [64]:
# Define the places of your data: 
from os import listdir
path = '/home/nils/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/'

files = ['file://' + path + f for f in listdir(path)]
files

['file:///home/nils/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19600101-19691231.nc',
 'file:///home/nils/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19900101-19991231.nc',
 'file:///home/nils/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19700101-19791231.nc',
 'file:///home/nils/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cmip5/cmip5/output1/MPI-M/MPI-ESM-LR/historical/day/atmos/day/r1i1p1/v20111006/psl/psl_day_MPI-ESM-LR_historical_r1i1p1_19800101-19891231.nc',
 'file:///home/nils/.conda/envs/birdhouse/var/li

In [68]:
projection = wps.execute(
    identifier="weatherregimes_projection", #indices_clipping",
    inputs=[
       ("resource",files[0]),
       ("resource",files[1]),
       ("resource",files[2]),
       ("resource",files[3]),
       ("resource",files[4]),
       ("season",'DJF'),
       ("dat", training.processOutputs[0].reference),
       ("Rdat", training.processOutputs[2].reference),
       ("netCDF",training.processOutputs[3].reference)
           ],
    async=True)

In [69]:
monitorExecution(projection, sleepSecs=5)
print projection.getStatus()

for o in projection.processOutputs:
    print o.reference

ProcessSucceeded
http://localhost:8090/wpsoutputs/flyingpigeon/output_pca-a19a0e84-4207-11e6-9e3e-417988cdf122.dat
http://localhost:8090/wpsoutputs/flyingpigeon/Routput_graphic-a19a0e84-4207-11e6-9e3e-417988cdf122.pdf
http://localhost:8090/wpsoutputs/flyingpigeon/output_classification-a19a0e84-4207-11e6-9e3e-417988cdf122.Rdat
http://localhost:8090/wpsoutputs/flyingpigeon/output_netcdf-a19a0e84-4207-11e6-9e3e-417988cdf122.nc
